In [1]:
import os
import re
import json

import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

:0: UserWarning: You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [2]:
webmd_url = "https://www.webmd.com/"

class SideEffectsSpider(scrapy.Spider):
    os.chdir("/Users/gogrean/Documents/Insight_Fellowship/Research/Mental_Health/NHANES_Survey/data")
    name = "sideeffects"
    start_urls = [
        webmd_url,
    ]
    
    def parse(self, response):
        meds_df = pd.read_csv("brand_to_generic_drug_names.csv")
        meds = [m.lower() for m in list(meds_df["Brand Name"].values)]
        for med in meds:
            list_link = response.urljoin("/drugs/2/alpha/" + med[0] + "/" + med[0:2])
            yield scrapy.Request(list_link, callback=self.parse_list, meta = {'drug' : med})
    
    def parse_list(self, response):
        med = response.meta['drug']
        css_selector = 'div[class="drug-list-container"] a[href*="/' + med + '/"]'
        med_details_link = response.css(css_selector).css("a::attr(href)").extract_first()  
        yield scrapy.Request(response.urljoin(med_details_link), 
                             callback=self.parse_med, meta = {'drug' : med})
        
    def parse_med(self, response):
        med = response.meta['drug']
        css_selector = 'a[data-metrics-link="sidef"]'
        med_sideeffects_link = response.css(css_selector).css("a::attr(href)").extract_first()
        yield scrapy.Request(response.urljoin(med_sideeffects_link), 
                             callback=self.parse_sideeffects, meta = {'drug' : med})
    
    def parse_sideeffects(self, response):
        med = response.meta['drug']
        resp = response.css('div[class="drug-submonograph-sideeffects"] ul')
        sideeffects_lists = [r.css('li').extract() for r in resp]
        sideeffects = {'common': {}, 
                       'infrequent': {},
                       'rare': {}
                      }
        for k_i, k in enumerate(['common', 'infrequent', 'rare']):
            sideeffects[k]['severe'] = [re.search("\<li\>(.*)\<\/li\>", s).group(1).strip().lower()
                                        for s in sideeffects_lists[2*k_i]]
            sideeffects[k]['less severe'] = [re.search("\<li\>(.*)\<\/li\>", s).group(1).strip().lower()
                                             for s in sideeffects_lists[2*k_i+1]]
        fname = "side_effects/" + med + "-sideeffects.json"
        with open(fname, 'w') as f:
            json.dump(sideeffects, f)
        

process = CrawlerProcess()

process.crawl(SideEffectsSpider)
process.start()

2018-01-16 11:55:19 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-01-16 11:55:19 [scrapy.utils.log] INFO: Versions: lxml 3.5.0.0, libxml2 2.9.2, cssselect 1.0.3, parsel 1.3.1, w3lib 1.18.0, Twisted 17.9.0, Python 3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) - [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)], pyOpenSSL 17.0.0 (OpenSSL 1.0.2g  1 Mar 2016), cryptography 1.7.1, Platform Darwin-17.3.0-x86_64-i386-64bit
2018-01-16 11:55:19 [scrapy.crawler] INFO: Overridden settings: {}
2018-01-16 11:55:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.logstats.LogStats']
2018-01-16 11:55:19 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

https://www.webmd.com/drugs/2/drug-8603/celexa-oral/details/list-sideeffects


2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/x/xa> (referer: https://www.webmd.com/)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-146745/invega-oral/details> (referer: https://www.webmd.com/drugs/2/alpha/i/in)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/n/ni> (referer: https://www.webmd.com/)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-164821/brisdelle-oral/details> (referer: https://www.webmd.com/drugs/2/alpha/b/br)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-1776/anafranil-oral/details/list-sideeffects> (referer: https://www.webmd.com/drugs/2/drug-1776/anafranil-oral/details)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-94034/rozerem-oral/deta

https://www.webmd.com/drugs/2/drug-1699/zyprexa-oral/details/list-sideeffects
https://www.webmd.com/drugs/2/drug-1776/anafranil-oral/details/list-sideeffects
https://www.webmd.com/drugs/2/drug-94034/rozerem-oral/details/list-sideeffects


2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/z/zo> (referer: https://www.webmd.com/)
2018-01-16 11:55:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/r/ri> (referer: https://www.webmd.com/)
2018-01-16 11:55:23 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.webmd.com/drugs/2/drug-7031/surmontil-oral/details> from <GET https://www.webmd.com/drugs/2/drug-7031-531/surmontil/details>
2018-01-16 11:55:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-64439/abilify-oral/details/list-sideeffects> (referer: https://www.webmd.com/drugs/2/drug-64439/abilify-oral/details)
2018-01-16 11:55:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/n/na> (referer: https://www.webmd.com/)
2018-01-16 11:55:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/alpha/o/or> (referer: ht

...

https://www.webmd.com/drugs/2/drug-168839/irenka-oral/details/list-sideeffects
https://www.webmd.com/drugs/2/drug-165442/versacloz-oral/details/list-sideeffects
https://www.webmd.com/drugs/2/drug-6936/norpramin-oral/details/list-sideeffects


2018-01-16 11:55:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-5194/clozaril-oral/details/list-sideeffects> (referer: https://www.webmd.com/drugs/2/drug-5194/clozaril-oral/details)


https://www.webmd.com/drugs/2/drug-5194/clozaril-oral/details/list-sideeffects


2018-01-16 11:55:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.webmd.com/drugs/2/drug-89420/fazaclo-oral/details/list-sideeffects> (referer: https://www.webmd.com/drugs/2/drug-89420/fazaclo-oral/details)
2018-01-16 11:55:30 [scrapy.core.engine] INFO: Closing spider (finished)
2018-01-16 11:55:30 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 80840,
 'downloader/request_count': 168,
 'downloader/request_method_count/GET': 168,
 'downloader/response_bytes': 3273670,
 'downloader/response_count': 168,
 'downloader/response_status_count/200': 136,
 'downloader/response_status_count/301': 32,
 'dupefilter/filtered': 83,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2018, 1, 16, 19, 55, 30, 70649),
 'log_count/DEBUG': 170,
 'log_count/ERROR': 16,
 'log_count/INFO': 7,
 'memusage/max': 148758528,
 'memusage/startup': 148754432,
 'request_depth_max': 3,
 'response_received_count': 136,
 'scheduler/dequeued': 168,
 'scheduler/

https://www.webmd.com/drugs/2/drug-89420/fazaclo-oral/details/list-sideeffects
